In [1]:
import library as lib
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import math

C:\Users\cdipt\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\cdipt\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\cdipt\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\cdipt\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
def g(x):
    return math.exp(-1*x)

In [55]:
def fixed_point_single_variable(g, guess, eps = 1e-4):
    """
    Function to find root using fixed point method for equation of a single variable

    args:
    g: The function g(x), such that g(x) = x
    guess: The guess values to start working with
    eps: The minimum error before the roots are returned
    """
    
    del_ = 1e5
    while(del_ > eps):
        guess_new = g(guess)
        del_ = abs(guess - guess_new)
        guess = guess_new
    return guess

def fixed_point_multi_variable(G, X, eps = 1e-4):
    """
    Function to find root using dixed point method for a system of equation
    
    args:
    G: list containing all the functions
    X: list containing initial guess values
    eps: The minimum error before the roots are returned
    """
    
    del_ = 1e5
    
    while del_ > eps:
        X_new = list(G[i](X) for i in range(len(X)))
        del_ = sum(list(abs(X_new[i] - X[i]) for i in range(len(X))))/len(X)
        X = X_new.copy()
    return X

In [8]:
def g1(X):
    return math.sqrt(10-X[0]*X[1])
def g2(X):
    return math.sqrt((57-X[0])/(3*X[1]))

In [9]:
X = [1.5,3.5]
G = [g1,g2]

In [2]:
def newton_raphson_multivariable(F, J, x0, eps = 1e-4):
    """This function is used to find the root of a given function using Newton-Raphson method.
    
    Parameters:
    - F: The list containing all the functions of the system 
    - J: The Jacobian matrix of f of type library.matrix.
    - x0: The list of initial guess.
    - eps: The tolerance.
    
    """
    x = x0.copy()
    iter = 0
    while True:
        
        J_val = lib.matrix.matrix_init((2,2), scheme="zero")
        for row in range(J.num_rows):
            for col in range(J.num_cols):
                J_val.matrix[row][col] = J.matrix[row][col](x)
        
        F_val = lib.matrix.matrix(list([F[i](x)] for i in range(len(F))))
        J_val_inv = lib.matrix.matrix(np.linalg.inv(np.array(J_val.matrix)).tolist())
        x_p = lib.matrix.matmul(J_val_inv,F_val).get_column(0)
        x = list(x[i]-x_p[i] for i in range(len(x)))
        print(x)
        iter += 1
        if sum(list(abs(F[i](x)) for i in range(len(x)))) < eps:
            break
    return x, iter

def jacobian(f, x, h=1e-5):
    """This function is used to find the Jacobian matrix of a system of functions.
    
    Parameters:
    - f: The system of functions.
    - x: The point at which the Jacobian matrix is to be evaluated.
    - h: The step size.
    
    Returns:
    - J: The Jacobian matrix of f.
    """
    n = len(f(x))
    m = len(x)
    J = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            x1 = x.copy()
            x2 = x.copy()
            x1[j] += h
            x2[j] -= h
            J[i, j] = (f(x1)[i] - f(x2)[i]) / (2 * h)
    return J

# Example usage:
# f = lambda x: np.array([x[0]**2 + x[1]**2 - 1, x[0] - x[1]])
# J = lambda x: np.array([[2*x[0], 2*x[1]], [1, -1]])
# x0 = np.array([0.5, 0.5])
# tol = 1e-5
# root, iter = newton_raphson_multivariable(f, J, x0, tol)
# solution should be [0.70710678, 0.70710678]
# print(root, iter)

In [3]:
def g1(X):
    return X[0]**2 + X[1]**2 - 1
def g2(X):
    return X[0] - X[1]
def gp11(X):
    return 2*X[0]
def gp12(X):
    return 2*X[1]
def gp21(X):
    return 1
def gp22(X):
    return -1

F = [g1,g2]
J = lib.matrix.matrix([[gp11,gp12],[gp21,gp22]])
x0 = [0.5, 0.5]

In [4]:
root, iter = newton_raphson_multivariable(F, J, x0)

[0.75, 0.75]
[0.7083333333333334, 0.7083333333333334]
[0.7071078431372549, 0.7071078431372549]


In [26]:
def gauss_nodes_and_weights(n):
    """
    This function is used to find the nodes and weights for Gaussian Quadrature.

    n: number of intervals.
        
    """
    x = []
    w = []
    if n == 1:
        x.append(0.0)
        w.append(2.0)
        return x, w
    for i in range(n):
        x0 = math.cos(math.pi * (i + 0.75) / (n + 0.5))
        x1 = 0
        while abs(x0 - x1) > 1e-10:
            p0 = 1.0
            p1 = 0.0
            for j in range(n):
                p2 = p1
                p1 = p0
                p0 = ((2 * j + 1) * x0 * p1 - j * p2) / (j + 1)
            pp = n * (x0 * p0 - p1) / (x0 * x0 - 1)
            x1 = x0
            x0 = x1 - p0 / pp
        x.append(x0)
        w.append(2 / ((1 - x0 * x0) * pp * pp))
    return x, w

def integration_using_Gauss_Quadrature(f, a, b, n):
    """
    Function to calculate the integral of a function using the Gaussian Quadrature Techmique
    
    f: The function whose integral has to be calculated
    a: The lower limit of integration
    b: The upper limit of integration
    n: The number of sample points for integration
    
    """
    int_ = 0
    x, w = gauss_nodes_and_weights(n)
    for i in range(n):
        int_ = int_ + w[i]*f(x[i])
    return int_

In [31]:
# Semi-implicit euler
def semi_implicit_euler(f,g, x0, v0, n, t_range):
    """
    The function to calculate the solution using semi-implicit Euler.
    """
    
    v = []
    x = []
    t0,tn = t_range
    
    dt = (tn-t0)/n
    
    v.append(v0)
    x.append(x0)
    t = 0
    for i in range(n):
        v.append(v[i] + g(t,x[i])*dt)
        x.append(x[i] + f(t,v[i+1])*dt)
        t = t+dt
    return x,v,t
        

In [32]:
def verlet(a, x0, v0, n, t_range):
    
    v = []
    x = []
    t0,tn = t_range
    dt = (tn-t0)/n
    
    x.append(x0)
    x.append(x0 + v0*dt + 0.5*a(x0)*(dt)**2)
    
    for i in range(n-1):
        x.append(2*x[-1] - x[-2] + a(x[-1])*(dt)**2)
    return x

In [34]:
def velocity_verlet(a, x0, v0, n, t_range):
    
    v = []
    x = []
    t0,tn = t_range
    dt = (tn-t0)/n
    
    x.append(x0)
    v.append(v0)
    
    for i in range(n):
        v_half_step = v[i] + 0.5*dt*a(x[i])
        x.append(x[i]+v_half_step*dt)
        v.append(v_half_step + 0.5*dt*a(x[i+1]))
    return x,v
    